<a href="https://colab.research.google.com/github/shubhu1026/AI-ML/blob/main/NLP_Midterm_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
import random

In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load dataset (assuming a CSV with "text" and "sentiment" columns)
df = pd.read_csv("/content/train_data.csv")

df.head()

,sentence,sentiment
0,awww that s a bummer you shoulda got david car...,0.0
1,is upset that he can t update his facebook by ...,0.0
2,i dived many times for the ball managed to sav...,0.0
3,my whole body feels itchy and like its on fire,0.0
4,no it s not behaving at all i m mad why am i h...,0.0


In [ ]:
df = df.rename(columns={"sentence": "text"})

In [ ]:
df.head()

,text,sentiment
0,awww that s a bummer you shoulda got david car...,0.0
1,is upset that he can t update his facebook by ...,0.0
2,i dived many times for the ball managed to sav...,0.0
3,my whole body feels itchy and like its on fire,0.0
4,no it s not behaving at all i m mad why am i h...,0.0


In [ ]:
# Preprocess function
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    tokens = word_tokenize(text)
    return text

In [ ]:
import numpy as np
from multiprocessing import Pool

# Parallelize clean_text function
def parallel_clean_text(df_subset):
    return df_subset["text"].apply(clean_text).tolist()

# Split data into chunks and process in parallel
def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    results = pool.map(func, df_split)
    pool.close()
    pool.join()
    return results

In [ ]:
df = df.sample(frac=0.05, random_state=42)

In [ ]:
df.shape

(76199, 2)

In [ ]:
df['sentiment'].dtype

dtype('int64')

(0 = negative, 1 = positive)

In [ ]:
positive_df = df[df["sentiment"] == 1]
negative_df = df[df["sentiment"] == 0]

In [ ]:
positive_df.shape, negative_df.shape

((38140, 2), (38059, 2))

In [ ]:
# Separate text based on sentiment
positive_texts = positive_df["text"].apply(clean_text).tolist()
negative_texts = negative_df["text"].apply(clean_text).tolist()

# positive_texts = parallelize_dataframe(df[df["sentiment"] == 0], parallel_clean_text)
# negative_texts = parallelize_dataframe(df[df["sentiment"] == 1], parallel_clean_text)

In [ ]:
print("First few positive texts:")
print(positive_texts[:5])  # Adjust the number (5) as needed

print("\nFirst few negative texts:")
print(negative_texts[:5])

First few positive texts:
['i sent comments of love to her l', 'haha o same shit still', 'playing around with icons joe pup needs one so', 'ight bed right now mass in the morning day spent working on the sites then the gym and dinner with the fam bam goodnight everyone', 'it was nice whole family was here lots of quality time how was yours']

First few negative texts:
['me too i think nanny should make me dinner now', 'dah they aren t on mine either my twitpic isn t working so i tried sending it to the thing it says to from the site wtf', 'omg i need sleep but i work at', 'im sorry im such a loser but only really ace people laugh at their own jokes nukka lmao ok sooo ugh i forgot now', 'such a shame you have to spend time and energy chasing these rippers and copycats']


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Combine all positive and negative texts into a single string
positive_text = " ".join(positive_texts)
negative_text = " ".join(negative_texts)

# Generate word cloud for positive sentiment
positive_wordcloud = WordCloud(width=800, height=400, background_color="white").generate(positive_text)

# Generate word cloud for negative sentiment
negative_wordcloud = WordCloud(width=800, height=400, background_color="white").generate(negative_text)

# Plot word clouds
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(positive_wordcloud, interpolation="bilinear")
plt.title("Positive Sentiment Word Cloud")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(negative_wordcloud, interpolation="bilinear")
plt.title("Negative Sentiment Word Cloud")
plt.axis("off")

plt.show()

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf_vectorizer = TfidfVectorizer()

# # Fit and transform the positive texts
# positive_tfidf = tfidf_vectorizer.fit_transform(positive_texts)

# # Fit and transform the negative texts
# negative_tfidf = tfidf_vectorizer.fit_transform(negative_texts)

In [ ]:
# # You can then generate responses based on sentiment
# response = generate_response("positive", {"positive": {"pi": pi, "A1": A1, "A2": A2, "A3": A3}})
# print("Chatbot:", response)

# response = generate_response("negative", {"negative": {"pi": pi, "A1": A1, "A2": A2, "A3": A3}})
# print("Chatbot:", response)

Chatbot: so i cant update twitter the way i used it after all thanks for following james looking forward to the weekend hehe
Chatbot: morning hon finished all my tests on thursday yet i still remain held captive by indy s spirit


In [ ]:
def fill_dict(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)

In [ ]:
def sample_word(d):
    p0 = np.random.random()
    cumulative = 0
    for t, p in d.items():
        cumulative += p
        if p0 < cumulative:
            return t

In [ ]:
def list_to_probdict(tokens):
    d = {}
    T = len(tokens)
    for t in tokens:
        d[t] = d.get(t, 0.) + 1
    for t, c in d.items():
        d[t] = c / T
    return d

In [ ]:
pi = {}  # Initial distribution
A1 = {}  # First order transition
A2 = {}  # Second order transition
A3 = {}  # Third order transition

# Loop through the DataFrame
for text in positive_texts:
        # Split the cleaned text into tokens
        tokens = text.split()
        T = len(tokens)
        if T > 2:
            # Update initial distribution using the first token
            pi[tokens[0]] = pi.get(tokens[0], 0) + 1

            # Build transitions for each token in the sequence
            for i in range(1, T):
                if i == 1:
                    fill_dict(A1, tokens[i-1], tokens[i])
                elif i == 2:
                    fill_dict(A2, (tokens[i-2], tokens[i-1]), tokens[i])
                else:
                    fill_dict(A3, (tokens[i-3], tokens[i-2], tokens[i-1]), tokens[i])
            # Add an end marker for the sentence
            fill_dict(A3, (tokens[-3], tokens[-2], tokens[-1]), "<end>")

# Normalize pi (initial distribution)
pi_total = sum(pi.values())
for t, count in pi.items():
    pi[t] = count / pi_total

# Normalize A1, A2, and A3
for t_1, token in A1.items():
    A1[t_1] = list_to_probdict(token)

for (t_2, t_1), token in A2.items():
    A2[(t_2, t_1)] = list_to_probdict(token)

for (t_3, t_2, t_1), token in A3.items():
    A3[(t_3, t_2, t_1)] = list_to_probdict(token)

In [ ]:
# Initialize negative Markov model dictionaries
pi_neg = {}  # Initial distribution for negative texts
A1_neg = {}  # First order transitions for negative texts
A2_neg = {}  # Second order transitions for negative texts
A3_neg = {}  # Third order transitions for negative texts

# Loop through the negative_texts list and build the model
for text in negative_texts:
    # Split the cleaned text into tokens (assuming text is already cleaned)
    tokens = text.split()
    T = len(tokens)

    if T > 2:
        # Update initial distribution using the first token
        pi_neg[tokens[0]] = pi_neg.get(tokens[0], 0) + 1

        # Build transitions for each token in the sequence
        for i in range(1, T):
            if i == 1:
                fill_dict(A1_neg, tokens[i-1], tokens[i])
            elif i == 2:
                fill_dict(A2_neg, (tokens[i-2], tokens[i-1]), tokens[i])
            else:
                fill_dict(A3_neg, (tokens[i-3], tokens[i-2], tokens[i-1]), tokens[i])

        # Add an end marker for the sentence
        fill_dict(A3_neg, (tokens[-3], tokens[-2], tokens[-1]), "<end>")

# Normalize the initial distribution for negative texts
pi_total_neg = sum(pi_neg.values())
for t, count in pi_neg.items():
    pi_neg[t] = count / pi_total_neg

# Normalize the transition dictionaries for negative texts
for t_1, token in A1_neg.items():
    A1_neg[t_1] = list_to_probdict(token)

for (t_2, t_1), token in A2_neg.items():
    A2_neg[(t_2, t_1)] = list_to_probdict(token)

for (t_3, t_2, t_1), token in A3_neg.items():
    A3_neg[(t_3, t_2, t_1)] = list_to_probdict(token)

In [ ]:
def generate_positive_sentence():
    sentence = []
    w0 = sample_word(pi)
    sentence.append(w0)

    if w0 not in A1:
        return " ".join(sentence)

    w1 = sample_word(A1[w0])
    sentence.append(w1)

    if (w0, w1) not in A2:
        return " ".join(sentence)

    w2 = sample_word(A2[(w0, w1)])
    sentence.append(w2)

    while True:
        if (w0, w1, w2) in A3:
            w3 = sample_word(A3[(w0, w1, w2)])
        elif (w1, w2) in A2:
            w3 = sample_word(A2[(w1, w2)])
        elif w2 in A1:
            w3 = sample_word(A1[w2])
        else:
            break
        if w3 == "<end>":
            break
        sentence.append(w3)
        w0, w1, w2 = w1, w2, w3

    return " ".join(sentence)

# Markov model sentence generation for negative sentiment
def generate_negative_sentence():
    sentence = []
    w0 = sample_word(pi_neg)
    sentence.append(w0)

    if w0 not in A1_neg:
        return " ".join(sentence)

    w1 = sample_word(A1_neg[w0])
    sentence.append(w1)

    if (w0, w1) not in A2_neg:
        return " ".join(sentence)

    w2 = sample_word(A2_neg[(w0, w1)])
    sentence.append(w2)

    while True:
        if (w0, w1, w2) in A3_neg:
            w3 = sample_word(A3_neg[(w0, w1, w2)])
        elif (w1, w2) in A2_neg:
            w3 = sample_word(A2_neg[(w1, w2)])
        elif w2 in A1_neg:
            w3 = sample_word(A1_neg[w2])
        else:
            break
        if w3 == "<end>":
            break
        sentence.append(w3)
        w0, w1, w2 = w1, w2, w3

    return " ".join(sentence)

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Define the sentiment classification function
def get_sentiment(text):
    score = sia.polarity_scores(text)
    if score["compound"] >= 0.05:
        return "positive"
    elif score["compound"] <= -0.05:
        return "negative"
    return None

In [ ]:
def generate_response(user_input):
    sentiment = get_sentiment(user_input)

    if sentiment == "positive":
        return generate_positive_sentence()
    elif sentiment == "negative":
        return generate_negative_sentence()
    else:
        return "It seems you're feeling neutral. Here's something to think about!"

Generated sentence: lol i wouldn t give for one night of un interrupted blissful sleep


In [ ]:
def chat():
    print("Chatbot: Hi there! How can I assist you today?")
    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Chatbot: Goodbye! Have a great day!")
            break

        response = generate_response(user_input)
        print("Chatbot:", response)

# Run the chatbot
chat()